In [ ]:
import requests
import pandas as pd

# input username, region code, and your api key here. These values are used as defaults for the lookup
gameName ='Rod Pole'
tagLine = 'NA1'
api_key = ''

#use the players puuid to get their name and region code
def get_name_and_tag(puuid=None, api_key=None):
    link = f'https://americas.api.riotgames.com/riot/account/v1/accounts/by-puuid/{puuid}?api_key={api_key}'

    response = requests.get(link)

    gameName = response.json()['gameName']
    tagLine = response.json()['tagLine']

    return f'{gameName}#{tagLine}'

#testing get_name_and_tag

# change the values after the equal sign to lookup someone other than Rod Pole
get_name_and_tag(puuid='tDtpfhaVeb86jupQ0p2qbBfa_EcPA6pTB_upRdZGDrT30MD1qZ3XRpF61bp3GSXGNXN_I_uofulOtw', api_key=api_key)

In [9]:
def get_puuid(summonerId=None, gameName=None, tagLine=None, region='americas'):
    """Gets the puuid from a summonerId or Riot_id and riot_tag
    
    Args:
        summonerId (str, optional): Summoner ID. Defaults to None.
        gameName (str, optional): Riot ID. Defaults to None.
        tagLine (str, optional): Riot Tag. Defaults to None.
        region (str, optional): Region. Defaults to 'americas'.
        
    Returns:
        str: puuid
    """

    if summonerId is not None:
        root_url = f'https://{region}.api.riotgames.com/'
        endpoint = 'lol/summoner/v4/summoners/'
        print(root_url+endpoint+summonerId+'?api_key='+api_key)
        response = requests.get(root_url+endpoint+summonerId+'?api_key='+api_key)

        return response.json()['puuid']
    else:
        root_url = f'https://{region}.api.riotgames.com/'
        endpoint = f'riot/account/v1/accounts/by-riot-id/{gameName}/{tagLine}'

        response = requests.get(root_url+endpoint+'?api_key='+api_key)

        return response.json()['puuid']
    
def get_idtag_from_puuid(puuid=None):
    """Gets the riot_id and riot_tag from a puuid
    
    Args:
        puuid (str, optional): puuid. Defaults to None.
        
    Returns:
        id (dict): Dictionary with riot_id and riot_tag.
    """
    root_url = 'https://americas.api.riotgames.com/'
    endpoint = 'riot/account/v1/accounts/by-puuid/'

    response = requests.get(root_url+endpoint+puuid+'?api_key='+api_key)

    id = {
        'gameName': response.json()['gameName'],
        'tagLine': response.json()['tagLine']
    }

    return id

In [10]:
#test get puuid function
get_puuid(gameName='Rod Pole', tagLine='NA1')

'tDtpfhaVeb86jupQ0p2qbBfa_EcPA6pTB_upRdZGDrT30MD1qZ3XRpF61bp3GSXGNXN_I_uofulOtw'

In [11]:
#test id function
get_idtag_from_puuid('tDtpfhaVeb86jupQ0p2qbBfa_EcPA6pTB_upRdZGDrT30MD1qZ3XRpF61bp3GSXGNXN_I_uofulOtw')

{'gameName': 'Rod Pole', 'tagLine': 'NA1'}

In [12]:
def get_match_history(region=None, puuid=None, start=0, count=20):
    root_url = f'https://{region}.api.riotgames.com/'
    endpoint = f'lol/match/v5/matches/by-puuid/{puuid}/ids'
    query_params = f'?start={start}&count={count}' #changing starting index (start) and number of matches retrieved (count)

    response = requests.get(root_url + endpoint + query_params + '&api_key=' + api_key)
    
    return response.json()

def get_match_data_from_id(region = None, matchId = None):

    root_url = f'https://{region}.api.riotgames.com/'
    endpoint = f'lol/match/v5/matches/{matchId}'


    response = requests.get(root_url + endpoint + '?api_key=' + api_key)

    return response.json()

In [13]:
#test get_match_history
get_match_history(region='americas', puuid='tDtpfhaVeb86jupQ0p2qbBfa_EcPA6pTB_upRdZGDrT30MD1qZ3XRpF61bp3GSXGNXN_I_uofulOtw', start=0, count=5)

['NA1_5121942380',
 'NA1_5111346226',
 'NA1_5111325432',
 'NA1_5111312830',
 'NA1_5111294215']

In [14]:
#test match data function. Use this url, and the middle option to understand output: https://developer.riotgames.com/apis#match-v5/GET_getMatch
game = get_match_data_from_id(region='americas', matchId='NA1_5111346226')

In [15]:
puuid = 'tDtpfhaVeb86jupQ0p2qbBfa_EcPA6pTB_upRdZGDrT30MD1qZ3XRpF61bp3GSXGNXN_I_uofulOtw'

def process_match_json(match_json, puuid):
    ## Architecture
    metadata = match_json['metadata']
    info = match_json['info']
    players = info['participants']
    participants = metadata['participants']
    teams = info['teams']
    player = players[participants.index(puuid)]
    perks = player['perks']
    stats = perks ['statPerks']
    styles = perks['styles']

    primary = styles[0]
    secondary = styles[1]

    match_id = metadata['matchId']


    game_creation = info['gameCreation']
    game_duration = info['gameDuration']
    game_start_timestamp = info['gameStartTimestamp']
    game_end_timestamp = info['gameEndTimestamp']
    patch = info['gameVersion']

    riot_id = player['riotIdGameName']
    riot_tag = player['riotIdTagline']
    summoner_id = player['summonerId']
    summoner_name = player['summonerName']

    win = player['win']

    champ_id = player['championId']
    champ_transform = player['championTransform']
    champ_level = player['champLevel']

    gold_earned = player['goldEarned']
    neutral_minions_killed = player['neutralMinionsKilled']
    total_minions_killed = player['totalMinionsKilled']

    kills = player['kills']
    deaths = player['deaths']
    assists = player['assists']
    first_blood = player['firstBloodKill']

    total_damage_dealt= player['totalDamageDealtToChampions']
    total_damage_shielded = player['totalDamageShieldedOnTeammates']
    total_damage_taken = player['totalDamageTaken']
    total_damage_healed = player['totalHealsOnTeammates']
    total_minions_killed = player['totalMinionsKilled']
    total_time_cc_dealt = player['totalTimeCCDealt']

    early_surrender = player['gameEndedInEarlySurrender']
    surrender = player['gameEndedInSurrender']

    item0 = player['item0']
    item1 = player['item1']
    item2 = player['item2']
    item3 = player['item3']
    item4 = player['item4']
    item5 = player['item5']
    item6 = player['item6']

    summoner_1_id = player['summoner1Id']
    summoner_2_id = player['summoner2Id']


    wards_placed = player['wardsPlaced']
    vision_score = player['visionScore']







    defense = stats['defense']
    flex = stats['flex']
    offense = stats['offense']


    primary_style = primary['style']
    secondary_style = primary['style']

    primary_keystone = primary['selections'][0]['perk']
    primary_perk_1 = primary['selections'][1]['perk']
    primary_perk_2 = primary['selections'][2]['perk']
    primary_perk_3 = primary['selections'][3]['perk']

    secondary_perk_1 = secondary['selections'][0]['perk']
    secondary_perk_2 = secondary['selections'][1]['perk']


    objectives_stolen = player['objectivesStolen']
    objectives_stolen_assists = player['objectivesStolenAssists']

    ##what is this?
    detector_wars_placed = player['detectorWardsPlaced']
    vision_wards_bought = player['visionWardsBoughtInGame']
    role = player['role']

    matchDF = pd.DataFrame({
        'match_id': [match_id],
        'participants': [participants],
        'game_creation': [game_creation],
        'game_duration': [game_duration],
        'game_start_timestamp': [game_start_timestamp],
        'game_end_timestamp': [game_end_timestamp],
        'patch': [patch],
        'puuid': [puuid],
        'riot_id': [riot_id],
        'riot_tag': [riot_tag],
        'summoner_id': [summoner_id],
        'summoner_name': [summoner_name],
        'win': [win],
        'champion': [champ_id],
        'champion_transform': [champ_transform],
        'champion_level': [champ_level],
        'kills': [kills],
        'deaths': [deaths],
        'assists': [assists],
        'summoner1_id': [summoner_1_id],
        'summoner2_id': [summoner_2_id],
        'gold_earned': [gold_earned],
        'total_minions_killed': [total_minions_killed],
        'total_neutral_minions_killed': [neutral_minions_killed],
        'early_surrender': [early_surrender],
        'surrender': [surrender],
        'first_blood': [first_blood],
        'objectives_stolen': [objectives_stolen],
        'objectives_stolen_assists': [objectives_stolen_assists],
        'total_damage_dealt_champions': [total_damage_dealt],
        'total_damage_taken': [total_damage_taken],
        'total_damage_shielded_teammates': [total_damage_shielded],
        'total_time_crowd_controlled': [total_time_cc_dealt],
        'vision_score': [vision_score],
        'wards_placed': [wards_placed],
        'item0': [item0],
        'item1': [item1],
        'item2': [item2],
        'item3': [item3],
        'item4': [item4],
        'item5': [item5],
        'item6': [item6],
        'perk_keystone': [primary_keystone],
        'perk_primary_row_1': [primary_perk_1],
        'perk_primary_row_2': [primary_perk_2],
        'perk_primary_row_3': [primary_perk_3],
        'perk_secondary_row_1': [secondary_perk_1],
        'perk_secondary_row_2': [secondary_perk_2],
        'perk_primary_style': [primary_style],
        'perk_secondary_style': [secondary_style],
        'perk_shard_defense': [defense],
        'perk_shard_flex': [flex],
        'perk_shard_offense': [offense],
    })

    return matchDF


In [16]:
process_match_json(game, 'tDtpfhaVeb86jupQ0p2qbBfa_EcPA6pTB_upRdZGDrT30MD1qZ3XRpF61bp3GSXGNXN_I_uofulOtw')

,match_id,participants,game_creation,game_duration,game_start_timestamp,game_end_timestamp,patch,puuid,riot_id,riot_tag,...,perk_primary_row_1,perk_primary_row_2,perk_primary_row_3,perk_secondary_row_1,perk_secondary_row_2,perk_primary_style,perk_secondary_style,perk_shard_defense,perk_shard_flex,perk_shard_offense
0,NA1_5111346226,[M8x1csN-L7TbwTrJRI_ZOWxRbSByGH41rW0_skZTxCXN2...,1726255239595,1248,1726255287082,1726256535591,14.18.616.8335,tDtpfhaVeb86jupQ0p2qbBfa_EcPA6pTB_upRdZGDrT30M...,Rod Pole,NA1,...,8306,8345,8347,8473,8242,8300,8300,5001,5008,5008


In [17]:
match_ids = get_match_history(region='americas', puuid='tDtpfhaVeb86jupQ0p2qbBfa_EcPA6pTB_upRdZGDrT30MD1qZ3XRpF61bp3GSXGNXN_I_uofulOtw')

df = pd.DataFrame()
for match_id in match_ids:
    game = get_match_data_from_id(region='americas', matchId=match_id)
    matchDF = process_match_json(game, puuid='tDtpfhaVeb86jupQ0p2qbBfa_EcPA6pTB_upRdZGDrT30MD1qZ3XRpF61bp3GSXGNXN_I_uofulOtw')

    df = pd.concat([df, matchDF])

In [18]:
df

,match_id,participants,game_creation,game_duration,game_start_timestamp,game_end_timestamp,patch,puuid,riot_id,riot_tag,...,perk_primary_row_1,perk_primary_row_2,perk_primary_row_3,perk_secondary_row_1,perk_secondary_row_2,perk_primary_style,perk_secondary_style,perk_shard_defense,perk_shard_flex,perk_shard_offense
0,NA1_5121942380,[2ZOYTJ6OgofcvKPPxB1X4RB0MeBfiqhal30Pr0mudXDgV...,1727475261312,1077,1727475505535,1727476582834,14.19.621.6353,tDtpfhaVeb86jupQ0p2qbBfa_EcPA6pTB_upRdZGDrT30M...,Rod Pole,NA1,...,8463,8429,8451,8126,8106,8400,8400,5001,5001,5007
0,NA1_5111346226,[M8x1csN-L7TbwTrJRI_ZOWxRbSByGH41rW0_skZTxCXN2...,1726255239595,1248,1726255287082,1726256535591,14.18.616.8335,tDtpfhaVeb86jupQ0p2qbBfa_EcPA6pTB_upRdZGDrT30M...,Rod Pole,NA1,...,8306,8345,8347,8473,8242,8300,8300,5001,5008,5008
0,NA1_5111325432,[T5hdV4RKhTHD85hA6cVYOXUuu2i3_OR7B6RlfQgsD9UDJ...,1726253179173,1618,1726253204688,1726254823114,14.18.616.8335,tDtpfhaVeb86jupQ0p2qbBfa_EcPA6pTB_upRdZGDrT30M...,Rod Pole,NA1,...,8143,8138,8106,8236,8210,8100,8100,5001,5008,5008
0,NA1_5111312830,[wRcpALBQ0JR7mN9d73KZqc88nLod7DKo3t6F-feu3_Qgs...,1726251296775,1438,1726251342499,1726252781268,14.18.616.8335,tDtpfhaVeb86jupQ0p2qbBfa_EcPA6pTB_upRdZGDrT30M...,Rod Pole,NA1,...,8463,8473,8242,8410,8345,8400,8400,5001,5001,5005
0,NA1_5111294215,[FG0W_yByVEMmEw8tSCB7_YKAk3hgUPUPnSn3MSAxR6U2T...,1726249323389,1522,1726249372222,1726250894843,14.18.616.8335,tDtpfhaVeb86jupQ0p2qbBfa_EcPA6pTB_upRdZGDrT30M...,Rod Pole,NA1,...,8126,8138,8106,8226,8210,8100,8100,5001,5008,5008
0,NA1_5111279528,[IxzWs557A6hlczju0Kx_PJCXkQyVRia7SHCgpRQBP-z26...,1726247774996,986,1726248019008,1726249005626,14.18.616.8335,tDtpfhaVeb86jupQ0p2qbBfa_EcPA6pTB_upRdZGDrT30M...,Rod Pole,NA1,...,8401,8473,8242,8347,8306,8400,8400,5001,5001,5007
0,NA1_5111264277,[XspnZ4EV3rwPiLeBsOXHvjIfym2Hi98JBbzNZ7-PHdQm1...,1726245415240,1938,1726245526177,1726247464734,14.18.616.8335,tDtpfhaVeb86jupQ0p2qbBfa_EcPA6pTB_upRdZGDrT30M...,Rod Pole,NA1,...,8226,8210,8237,8473,8453,8200,8200,5001,5008,5007
0,NA1_5111242553,[v4qtsJvtlNTTCTPvsyC0wYpHbAxQIZGTVtdw9ibonP622...,1726242861154,1796,1726242896344,1726244692894,14.18.616.8335,tDtpfhaVeb86jupQ0p2qbBfa_EcPA6pTB_upRdZGDrT30M...,Rod Pole,NA1,...,8226,8210,8237,8473,8453,8200,8200,5001,5008,5007
0,NA1_5111230488,[aiEhN4m0CGNRmNnxcBx0Qn7Gf6LK3yT-9T4wKHqUWedcp...,1726240487284,2044,1726240534252,1726242579060,14.18.616.8335,tDtpfhaVeb86jupQ0p2qbBfa_EcPA6pTB_upRdZGDrT30M...,Rod Pole,NA1,...,9111,9105,8017,8429,8451,8000,8000,5001,5001,5005
0,NA1_5111219174,[Fw7wkwkavhhyR7ZFG5CRT5CcYWMJOYwUIr-MoDWckZLgZ...,1726238160624,1916,1726238222704,1726240139320,14.18.616.8335,tDtpfhaVeb86jupQ0p2qbBfa_EcPA6pTB_upRdZGDrT30M...,Rod Pole,NA1,...,8446,8429,8451,8345,8347,8400,8400,5001,5001,5007


In [19]:
perk = 'https://raw.communitydragon.org/latest/plugins/rcp-be-lol-game-data/global/default/v1/perks.json'
perk_styles = 'https://raw.communitydragon.org/latest/plugins/rcp-be-lol-game-data/global/default/v1/perkstyles.json'
champ = 'https://raw.communitydragon.org/latest/plugins/rcp-be-lol-game-data/global/default/v1/champion-summary.json'
item = 'https://raw.communitydragon.org/latest/plugins/rcp-be-lol-game-data/global/default/v1/items.json'

perk_json = requests.get(perk).json()
perk_styles_json = requests.get(perk_styles).json()
champ_json = requests.get(champ).json()
item_json = requests.get(item).json()

In [20]:
def json_extract(obj, key):
    arr = []
    
    def extract(obj, arr, key):
        if isinstance(obj, dict):
            for k, v in obj.items():
                if k == key:
                    arr.append(v)
                elif isinstance(v, (dict, list)):
                    extract(v, arr, key)
        elif isinstance(obj, list):
            for item in obj:
                extract(item, arr, key)
        return arr

    values = extract(obj, arr, key)
    return values


In [21]:
import pandas as pd



# Base URL for the champion icons
base_url = 'https://raw.communitydragon.org/latest/plugins/rcp-be-lol-game-data/global/default/v1/champion-icons/'

# Add a new column with the image URLs
df['image_url'] = df['champion'].apply(lambda x: f"{base_url}{x}.png")

# Display the DataFrame
print(df)


         match_id                                       participants  \
0  NA1_5121942380  [2ZOYTJ6OgofcvKPPxB1X4RB0MeBfiqhal30Pr0mudXDgV...   
0  NA1_5111346226  [M8x1csN-L7TbwTrJRI_ZOWxRbSByGH41rW0_skZTxCXN2...   
0  NA1_5111325432  [T5hdV4RKhTHD85hA6cVYOXUuu2i3_OR7B6RlfQgsD9UDJ...   
0  NA1_5111312830  [wRcpALBQ0JR7mN9d73KZqc88nLod7DKo3t6F-feu3_Qgs...   
0  NA1_5111294215  [FG0W_yByVEMmEw8tSCB7_YKAk3hgUPUPnSn3MSAxR6U2T...   
0  NA1_5111279528  [IxzWs557A6hlczju0Kx_PJCXkQyVRia7SHCgpRQBP-z26...   
0  NA1_5111264277  [XspnZ4EV3rwPiLeBsOXHvjIfym2Hi98JBbzNZ7-PHdQm1...   
0  NA1_5111242553  [v4qtsJvtlNTTCTPvsyC0wYpHbAxQIZGTVtdw9ibonP622...   
0  NA1_5111230488  [aiEhN4m0CGNRmNnxcBx0Qn7Gf6LK3yT-9T4wKHqUWedcp...   
0  NA1_5111219174  [Fw7wkwkavhhyR7ZFG5CRT5CcYWMJOYwUIr-MoDWckZLgZ...   
0  NA1_5111210004  [YwfEY54NdN4DH7FAFXCGz4ML0eGcSj8PagJ51yxFlhb06...   
0  NA1_5111197756  [DHl3LGBgzPnbVEoWfdnCusNwDUwsxCqjFDSRG9dt6EJZH...   
0  NA1_5111184327  [d5dLY3jPoedRfCi9eEBCEGs4utYRaoGrafye-2SKupH8

In [22]:
from IPython.display import display, Image

# Display all champion images from the DataFrame
for image_url in df['image_url']:
    display(Image(url=image_url))


In [23]:
# code to replace all of the perk numbers with the actual perks

perk_ids = json_extract(perk_json, 'id')
perk_names = json_extract(perk_json, 'name')

perk_dict = dict(map(lambda i, j : (int(i),j), perk_ids, perk_names))

perk_styles_ids = json_extract(perk_styles_json, 'id')
perk_styles_names = json_extract(perk_styles_json, 'name')

perk_styles_dict = dict(map(lambda i, j : (int(i),j), perk_styles_ids, perk_styles_names))

In [24]:
# code to replace all of the champion numbers with the actual champions
champ_ids = json_extract(champ_json, 'id')
champ_names = json_extract(champ_json, 'name')

# Assuming champ_json is a list of dictionaries with each dictionary containing 'id' and 'name'
champ_dict = {int(champ['id']): champ['name'] for champ in champ_json}



In [25]:
# code to replace all of the item numbers with the actual items
item_ids = json_extract(item_json, 'id')
item_names = json_extract(item_json, 'name')

# Assuming champ_json is a list of dictionaries with each dictionary containing 'id' and 'name'
item_dict = {int(item['id']): item['name'] for item in item_json}


In [26]:
pd.options.display.max_columns = 100

df.replace(perk_dict).replace(perk_styles_dict)

# Replace only the 'champion' column with the values from champ_dict
df['champion'] = df['champion'].replace(champ_dict)

# replace the item columns with the the actual items names from the item_dict
df['item0'] = df['item0'].replace(item_dict)
df['item1'] = df['item1'].replace(item_dict)
df['item2'] = df['item2'].replace(item_dict)
df['item3'] = df['item3'].replace(item_dict)
df['item4'] = df['item4'].replace(item_dict)
df['item5'] = df['item5'].replace(item_dict)
df['item6'] = df['item6'].replace(item_dict)
df


,match_id,participants,game_creation,game_duration,game_start_timestamp,game_end_timestamp,patch,puuid,riot_id,riot_tag,summoner_id,summoner_name,win,champion,champion_transform,champion_level,kills,deaths,assists,summoner1_id,summoner2_id,gold_earned,total_minions_killed,total_neutral_minions_killed,early_surrender,surrender,first_blood,objectives_stolen,objectives_stolen_assists,total_damage_dealt_champions,total_damage_taken,total_damage_shielded_teammates,total_time_crowd_controlled,vision_score,wards_placed,item0,item1,item2,item3,item4,item5,item6,perk_keystone,perk_primary_row_1,perk_primary_row_2,perk_primary_row_3,perk_secondary_row_1,perk_secondary_row_2,perk_primary_style,perk_secondary_style,perk_shard_defense,perk_shard_flex,perk_shard_offense,image_url
0,NA1_5121942380,[2ZOYTJ6OgofcvKPPxB1X4RB0MeBfiqhal30Pr0mudXDgV...,1727475261312,1077,1727475505535,1727476582834,14.19.621.6353,tDtpfhaVeb86jupQ0p2qbBfa_EcPA6pTB_upRdZGDrT30M...,Rod Pole,NA1,I9C_lDe3pIHZuqJ4Z6leoWhrx3I7G4OMB9nmuDr8feypCxU,Rod Pole,True,Amumu,0,17,1,16,30,4,6,10801,22,0,False,False,False,0,0,20399,31144,0,136,0,0,Malignance,Stormsurge,Sorcerer's Shoes,Hextech Alternator,Needlessly Large Rod,0,Poro-Snax,8439,8463,8429,8451,8126,8106,8400,8400,5001,5001,5007,https://raw.communitydragon.org/latest/plugins...
0,NA1_5111346226,[M8x1csN-L7TbwTrJRI_ZOWxRbSByGH41rW0_skZTxCXN2...,1726255239595,1248,1726255287082,1726256535591,14.18.616.8335,tDtpfhaVeb86jupQ0p2qbBfa_EcPA6pTB_upRdZGDrT30M...,Rod Pole,NA1,I9C_lDe3pIHZuqJ4Z6leoWhrx3I7G4OMB9nmuDr8feypCxU,Rod Pole,True,Thresh,0,10,2,2,6,4,14,6203,21,0,False,True,False,0,0,6305,10333,712,182,35,16,Mercury's Treads,Solstice Sleigh,Warmog's Armor,Ruby Crystal,0,0,Oracle Lens,8351,8306,8345,8347,8473,8242,8300,8300,5001,5008,5008,https://raw.communitydragon.org/latest/plugins...
0,NA1_5111325432,[T5hdV4RKhTHD85hA6cVYOXUuu2i3_OR7B6RlfQgsD9UDJ...,1726253179173,1618,1726253204688,1726254823114,14.18.616.8335,tDtpfhaVeb86jupQ0p2qbBfa_EcPA6pTB_upRdZGDrT30M...,Rod Pole,NA1,I9C_lDe3pIHZuqJ4Z6leoWhrx3I7G4OMB9nmuDr8feypCxU,Rod Pole,False,Gragas,0,12,3,7,3,4,14,7398,49,0,False,False,False,0,0,9061,17668,0,142,54,26,Luden's Companion,Zaz'Zak's Realmspike,Sorcerer's Shoes,Sheen,Aether Wisp,Amplifying Tome,Oracle Lens,8112,8143,8138,8106,8236,8210,8100,8100,5001,5008,5008,https://raw.communitydragon.org/latest/plugins...
0,NA1_5111312830,[wRcpALBQ0JR7mN9d73KZqc88nLod7DKo3t6F-feu3_Qgs...,1726251296775,1438,1726251342499,1726252781268,14.18.616.8335,tDtpfhaVeb86jupQ0p2qbBfa_EcPA6pTB_upRdZGDrT30M...,Rod Pole,NA1,I9C_lDe3pIHZuqJ4Z6leoWhrx3I7G4OMB9nmuDr8feypCxU,Rod Pole,False,Tahm Kench,0,9,3,6,5,4,3,7035,38,0,False,False,False,0,0,8547,26757,1107,144,52,21,Warmog's Armor,Celestial Opposition,Ruby Crystal,Mercury's Treads,Bramble Vest,0,Oracle Lens,8465,8463,8473,8242,8410,8345,8400,8400,5001,5001,5005,https://raw.communitydragon.org/latest/plugins...
0,NA1_5111294215,[FG0W_yByVEMmEw8tSCB7_YKAk3hgUPUPnSn3MSAxR6U2T...,1726249323389,1522,1726249372222,1726250894843,14.18.616.8335,tDtpfhaVeb86jupQ0p2qbBfa_EcPA6pTB_upRdZGDrT30M...,Rod Pole,NA1,I9C_lDe3pIHZuqJ4Z6leoWhrx3I7G4OMB9nmuDr8feypCxU,Rod Pole,True,Lux,0,14,3,1,9,4,14,9551,52,4,False,False,False,0,0,12226,10280,1133,222,55,21,Luden's Companion,Zaz'Zak's Realmspike,0,Hextech Alternator,Fiendish Codex,Sorcerer's Shoes,Oracle Lens,8128,8126,8138,8106,8226,8210,8100,8100,5001,5008,5008,https://raw.communitydragon.org/latest/plugins...
0,NA1_5111279528,[IxzWs557A6hlczju0Kx_PJCXkQyVRia7SHCgpRQBP-z26...,1726247774996,986,1726248019008,1726249005626,14.18.616.8335,tDtpfhaVeb86jupQ0p2qbBfa_EcPA6pTB_upRdZGDrT30M...,Rod Pole,NA1,I9C_lDe3pIHZuqJ4Z6leoWhrx3I7G4OMB9nmuDr8feypCxU,Rod Pole,False,Rell,0,8,0,5,4,4,14,3523,21,0,False,True,False,0,0,1756,9487,0,33,20,11,Plated Steelcaps,Celestial Opposition,Giant's Belt,Ruby Crystal,0,0,Oracle Lens,8439,8401,8473,8242,8347,8306,8400,8400,5001,5001,5007,https://raw.communitydragon.org/latest/plugins...
0,NA1_5111264277,[XspnZ4EV3rwPiLeBsOXHvjIfym2Hi98JB

In [27]:
# Replace the values based on the dictionaries and drop the first 12 columns
filtered_df = df.replace(perk_dict).replace(perk_styles_dict).iloc[:10, 12:]

#drop columns i dont want
filtered_df = filtered_df.drop(columns = ['champion_transform', 'total_neutral_minions_killed', 'early_surrender', 'surrender',  'first_blood', 'objectives_stolen',  'objectives_stolen_assists' ])

# Replace True with 'Win' and False with 'Loss' using map
# Replace True with 'Win' and False with 'Loss'
filtered_df['win'] = filtered_df['win'].replace({True: 'Win', False: 'Loss'})




# Output the result to a CSV file
filtered_df.to_csv('match_hist2.csv', index=False)

filtered_df


,win,champion,champion_level,kills,deaths,assists,summoner1_id,summoner2_id,gold_earned,total_minions_killed,total_damage_dealt_champions,total_damage_taken,total_damage_shielded_teammates,total_time_crowd_controlled,vision_score,wards_placed,item0,item1,item2,item3,item4,item5,item6,perk_keystone,perk_primary_row_1,perk_primary_row_2,perk_primary_row_3,perk_secondary_row_1,perk_secondary_row_2,perk_primary_style,perk_secondary_style,perk_shard_defense,perk_shard_flex,perk_shard_offense,image_url
0,Win,Amumu,17,1,16,30,4,6,10801,22,20399,31144,0,136,0,0,Malignance,Stormsurge,Sorcerer's Shoes,Hextech Alternator,Needlessly Large Rod,0,Poro-Snax,Aftershock,Font of Life,Conditioning,Overgrowth,Cheap Shot,Ultimate Hunter,Resolve,Resolve,Health Scaling,Health Scaling,Ability Haste,https://raw.communitydragon.org/latest/plugins...
0,Win,Thresh,10,2,2,6,4,14,6203,21,6305,10333,712,182,35,16,Mercury's Treads,Solstice Sleigh,Warmog's Armor,Ruby Crystal,0,0,Oracle Lens,Glacial Augment,Hextech Flashtraption,Biscuit Delivery,Cosmic Insight,Bone Plating,Unflinching,Inspiration,Inspiration,Health Scaling,Adaptive Force,Adaptive Force,https://raw.communitydragon.org/latest/plugins...
0,Loss,Gragas,12,3,7,3,4,14,7398,49,9061,17668,0,142,54,26,Luden's Companion,Zaz'Zak's Realmspike,Sorcerer's Shoes,Sheen,Aether Wisp,Amplifying Tome,Oracle Lens,Electrocute,Sudden Impact,Eyeball Collection,Ultimate Hunter,Gathering Storm,Transcendence,Precision,Precision,Health Scaling,Adaptive Force,Adaptive Force,https://raw.communitydragon.org/latest/plugins...
0,Loss,Tahm Kench,9,3,6,5,4,3,7035,38,8547,26757,1107,144,52,21,Warmog's Armor,Celestial Opposition,Ruby Crystal,Mercury's Treads,Bramble Vest,0,Oracle Lens,Guardian,Font of Life,Bone Plating,Unflinching,Approach Velocity,Biscuit Delivery,Resolve,Resolve,Health Scaling,Health Scaling,Attack Speed,https://raw.communitydragon.org/latest/plugins...
0,Win,Lux,14,3,1,9,4,14,9551,52,12226,10280,1133,222,55,21,Luden's Companion,Zaz'Zak's Realmspike,0,Hextech Alternator,Fiendish Codex,Sorcerer's Shoes,Oracle Lens,Dark Harvest,Cheap Shot,Eyeball Collection,Ultimate Hunter,Manaflow Band,Transcendence,Precision,Precision,Health Scaling,Adaptive Force,Adaptive Force,https://raw.communitydragon.org/latest/plugins...
0,Loss,Rell,8,0,5,4,4,14,3523,21,1756,9487,0,33,20,11,Plated Steelcaps,Celestial Opposition,Giant's Belt,Ruby Crystal,0,0,Oracle Lens,Aftershock,Shield Bash,Bone Plating,Unflinching,Cosmic Insight,Hextech Flashtraption,Resolve,Resolve,Health Scaling,Health Scaling,Ability Haste,https://raw.communitydragon.org/latest/plugins...
0,Win,Milio,14,2,2,16,4,3,8633,19,6690,12435,4983,252,79,38,Control Ward,Zaz'Zak's Realmspike,Shurelya's Battlesong,Moonstone Renewer,Ionian Boots of Lucidity,Kindlegem,Oracle Lens,Summon Aery,Manaflow Band,Transcendence,Scorch,Bone Plating,Revitalize,Sorcery,Sorcery,Health Scaling,Adaptive Force,Ability Haste,https://raw.communitydragon.org/latest/plugins...
0,Win,Lulu,14,2,5,19,4,3,8669,26,10893,19586,5200,353,76,36,Mikael's Blessing,Zaz'Zak's Realmspike,Shurelya's Battlesong,Ionian Boots of Lucidity,Control Ward,Kindlegem,Oracle Lens,Summon Aery,Manaflow Band,Transcendence,Scorch,Bone Plating,Revitalize,Sorcery,Sorcery,Health Scaling,Adaptive Force,Ability Haste,https://raw.communitydragon.org/latest/plugins...
0,Win,Bard,14,3,7,18,4,14,9499,29,12330,37023,0,271,103,43,Boots of Swiftness,Bloodsong,Frozen Heart,Kindlegem,Warmog's Armor,0,Oracle Lens,Fleet Footwork,Triumph,Legend: Haste,Cut Down,Conditioning,Overgrowth,Domination,Domination,Health Scaling,Health Scaling,Attack Speed,https://raw.communitydragon.org/latest/plugins...
0,Win,Blitzcrank,14,1,1,13,4,14,8852,34,9245,13500,992,389,89,45,Locket of the Iron Solari,Celestial Opposition,0,Boots of Swiftness,Kindlegem,Frozen Heart,Oracle Lens,Aftershock,Demolish,Conditioning,Overgrowth,Biscuit Delivery,Cosmic Insight,Resolve,Resolve,Health Scaling,Health Scaling,Ability Haste,https://raw.communitydragon.org/latest/plugins...
